<a href="https://colab.research.google.com/github/chiku27/Deploy_mini/blob/main/deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok

In [2]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import pandas as pd
from flask_ngrok import run_with_ngrok



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stop_words = set(stopwords.words('english'))
stemmer=SnowballStemmer('english')
def stem(text):
  text1 = [stemmer.stem(word) for word in text.split()]
  return " ".join(text1)

In [9]:
def remov_stops(text):
  text1=[word.lower() for word in text.split() if word.lower() not in stop_words]
  return  " ".join(text1)

def preprocessing(df):
  df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
  df=df.apply(stem)
  df=df.apply(remov_stops)
  return df

In [10]:
cols=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [11]:
df=pd.read_pickle('/content/gdrive/MyDrive/Mini_Project/new_clean_data1.pkl')
df_train=df[:1000]


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=4000)
X= cv.fit_transform(df_train['comment_text']).toarray()


In [39]:
def text_ip(val1):
  df_ip=pd.DataFrame()
  df_ip['text']=val1
  df_ip=df_ip.apply(preprocessing)
  return df_ip

In [45]:
app = Flask(__name__)
run_with_ngrok(app)
filename = '/content/gdrive/MyDrive/Mini_Project/NB_model'
model = pickle.load(open(filename, 'rb'))
@app.route('/')
def imp():
  return render_template('/content/gdrive/MyDrive/Mini_Project/templates/index.html')
@app.route('/predict',methods=['POST'])
def prediction_fun():
  inp_text=[str(x) for x in request.form.values()]
  dfx=text_ip(inp_text)
  X_1=cv.fit_transform(dfx).toarray()
  df_p={}
  for col in cols:
    model.fit(X,df_train[col])
    pred=model.predict_proba(X_1)[:,1]
    df_p[col]=pred.tolist()
    return render_template('/content/gdrive/MyDrive/Mini_Project/templates/index.html', prediction_text='Prediction probabilities are {}'.format(df_p))
if __name__=="__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2350ff5d7ee2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2020-12-12 07:42:30,582] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-45-5d3fb7a55093>", line 7, in imp
    return render_template('/content/gdrive/MyDrive/Mini_